In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [2]:
test = pd.read_csv('data/test.csv')
train = pd.read_csv('data/train.csv')

In [3]:
X_train = train.copy().drop(['target'], axis=1)
y_train = train['target']
X_test = test.copy()

* * *

### 결측치 최빈값 대체

In [4]:
cat = ['id','bin_0','bin_1','bin_2','bin_3','bin_4','nom_0','nom_1','nom_2','nom_3','nom_4','nom_5','nom_6','nom_7','nom_8','nom_9','ord_0','ord_1','ord_2','ord_3','ord_4','ord_5','day','month']

In [5]:
from sklearn.impute import SimpleImputer 

imp = SimpleImputer(strategy="most_frequent")
X_train[cat] = imp.fit_transform(X_train[cat])
X_test[cat] = imp.fit_transform(X_test[cat])

### bin_3, bin_4
binary encoding

In [ ]:
X_train['bin_3'].value_counts()

In [ ]:
import category_encoders as ce
BE = ce.binary.BinaryEncoder(['bin_3'])

X_train_be3 = BE.fit_transform(X_train['bin_3'], y_train)
X_test_be3 = BE.transform(X_test['bin_3'])

In [ ]:
X_train = pd.concat([X_train, X_train_be3], axis=1)
X_test = pd.concat([X_test, X_test_be3], axis=1)

In [ ]:
X_train.drop(['bin_3'], axis=1, inplace=True)
X_test.drop(['bin_3'], axis=1, inplace=True)

In [ ]:
X_train.info()

In [ ]:
X_train['bin_4'].value_counts()

In [ ]:
import category_encoders as ce
BE = ce.binary.BinaryEncoder(['bin_4'])

X_train_be4 = BE.fit_transform(X_train['bin_4'], y_train)
X_test_be4 = BE.transform(X_test['bin_4'])

In [ ]:
X_train = pd.concat([X_train, X_train_be4], axis=1)
X_test = pd.concat([X_test, X_test_be4], axis=1)

In [ ]:
X_train.drop(['bin_4'], axis=1, inplace=True)
X_test.drop(['bin_4'], axis=1, inplace=True)

In [ ]:
X_train.info()

### nom_0, nom_1, nom_2, nom_3, nom_4
label(ordinal) encoding

In [6]:
cat = ['nom_0', 'nom_1', 'nom_2', 'nom_3', 'nom_4']

In [7]:
from sklearn.preprocessing import OrdinalEncoder

X_train_oe, X_test_oe, = X_train.copy(), X_test.copy()

le = OrdinalEncoder(handle_unknown='use_encoded_value',
                    unknown_value=-1, dtype=int)

X_train_oe[cat] = le.fit_transform(X_train[cat])
X_test_oe[cat] = le.transform(X_test[cat])


X_train_oe[cat]

,nom_0,nom_1,nom_2,nom_3,nom_4
0,2,4,3,5,0
1,2,3,0,4,3
2,2,5,3,0,0
3,2,0,3,3,3
4,2,5,3,2,3
...,...,...,...,...,...
599995,2,1,0,4,3
599996,0,1,2,2,1
599997,2,0,0,5,3
599998,2,1,0,4,2


In [8]:
X_train[cat] = X_train_oe[cat]
X_test[cat] = X_test_oe[cat]

X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600000 entries, 0 to 599999
Data columns (total 24 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   id      600000 non-null  object
 1   bin_0   600000 non-null  object
 2   bin_1   600000 non-null  object
 3   bin_2   600000 non-null  object
 4   bin_3   600000 non-null  object
 5   bin_4   600000 non-null  object
 6   nom_0   600000 non-null  int32 
 7   nom_1   600000 non-null  int32 
 8   nom_2   600000 non-null  int32 
 9   nom_3   600000 non-null  int32 
 10  nom_4   600000 non-null  int32 
 11  nom_5   600000 non-null  object
 12  nom_6   600000 non-null  object
 13  nom_7   600000 non-null  object
 14  nom_8   600000 non-null  object
 15  nom_9   600000 non-null  object
 16  ord_0   600000 non-null  object
 17  ord_1   600000 non-null  object
 18  ord_2   600000 non-null  object
 19  ord_3   600000 non-null  object
 20  ord_4   600000 non-null  object
 21  ord_5   600000 non-null  object
 

### nom_5, nom_6, nom_7

target encoding

In [9]:
cat = ['nom_5', 'nom_6', 'nom_7']

In [ ]:
from category_encoders import TargetEncoder


X_train_te, X_test_te, = X_train.copy(), X_test.copy()

te = TargetEncoder(cols=cat, min_samples_leaf=5, smoothing=30) 
X_train_te[cat] = te.fit_transform(X_train[cat], y_train)
X_test_te[cat] = te.transform(X_test[cat])


X_train_te[cat]

In [ ]:
X_train[cat] = X_train_oe[cat]
X_test[cat] = X_test_oe[cat]

X_train.info()

### nom_6

In [ ]:
X_train['nom_6'].value_counts()

In [ ]:
# Frequency Encoding
fe = X_train.groupby("nom_6").size()/len(X_train)
X_train.loc[:, "nom_6_freq_encode"] = X_train["nom_6"].map(fe)

In [ ]:
#여기 잘 모르겠음 일단 test를 train 기준으로 변환하겠음
X_test.loc[:, "nom_6_freq_encode"] = X_test["nom_6"].map(fe)

### nom_7

In [ ]:
X_train['nom_7'].value_counts()

In [ ]:
# Frequency Encoding
fe = X_train.groupby("nom_7").size()/len(X_train)
X_train.loc[:, "nom_7_freq_encode"] = X_train["nom_7"].map(fe)

In [ ]:
#여기 잘 모르겠음 일단 test를 train 기준으로 변환하겠음
X_test.loc[:, "nom_7_freq_encode"] = X_test["nom_7"].map(fe)

### nom_8

In [ ]:
X_train['nom_8'].value_counts()

In [ ]:
# Frequency Encoding
fe = X_train.groupby("nom_8").size()/len(X_train)
X_train.loc[:, "nom_8_freq_encode"] = X_train["nom_8"].map(fe)

In [ ]:
#여기 잘 모르겠음 일단 test를 train 기준으로 변환하겠음
X_test.loc[:, "nom_8_freq_encode"] = X_test["nom_8"].map(fe)

### nom_9

In [ ]:
X_train['nom_9'].value_counts()

In [ ]:
# Frequency Encoding
fe = X_train.groupby("nom_9").size()/len(X_train)
X_train.loc[:, "nom_9_freq_encode"] = X_train["nom_9"].map(fe)

In [ ]:
#여기 잘 모르겠음 일단 test를 train 기준으로 변환하겠음
X_test.loc[:, "nom_9_freq_encode"] = X_test["nom_9"].map(fe)

### ord_1

In [ ]:
X_train['ord_1'].value_counts()

In [ ]:
train[train['target'] == 1]['ord_1'].value_counts()

- 실력차이에 따라 target이 바뀌는듯 하다. Weight of Evidence Encoding 

In [ ]:
# 각 범주가 target = 1일 확률(좋은(Good) = 1일 확률)을 계산한다.
woe_df = train.groupby('ord_1')['target'].mean()
woe_df = pd.DataFrame(woe_df)
# 칼럼의 이름을 "Good"으로 바꾸어 좀 더 이해하기 쉽게 한다.
woe_df = woe_df.rename(columns = {'target':'good'})
# "Bad"는 "Good"의 확률에 반대되는 부분의 확률이다.
woe_df['bad'] = 1-woe_df.good
# 분모에 최소한의 값을 더하여 0으로 나뉘는 일을 막는다.
woe_df['bad'] = np.where(woe_df['bad'] == 0, 1e-6, woe_df['bad'])
# WoE를 계산한다. 
woe_df['WoE'] = np.log(woe_df.good/woe_df.bad)
woe_df

In [ ]:
X_train.loc[:, 'ord_1_WoE_Encode'] = X_train['ord_1'].map(woe_df['WoE'])

In [ ]:
X_test.loc[:, 'ord_1_WoE_Encode'] = X_train['ord_1'].map(woe_df['WoE'])

### ord_2

In [ ]:
X_train['ord_2'].value_counts()

In [ ]:
train[train['target'] == 1]['ord_2'].value_counts()

In [ ]:
# 각 범주가 target = 1일 확률(좋은(Good) = 1일 확률)을 계산한다.
woe_df = train.groupby('ord_2')['target'].mean()
woe_df = pd.DataFrame(woe_df)
# 칼럼의 이름을 "Good"으로 바꾸어 좀 더 이해하기 쉽게 한다.
woe_df = woe_df.rename(columns = {'target':'good'})
# "Bad"는 "Good"의 확률에 반대되는 부분의 확률이다.
woe_df['bad'] = 1-woe_df.good
# 분모에 최소한의 값을 더하여 0으로 나뉘는 일을 막는다.
woe_df['bad'] = np.where(woe_df['bad'] == 0, 1e-6, woe_df['bad'])
# WoE를 계산한다. 
woe_df['WoE'] = np.log(woe_df.good/woe_df.bad)
woe_df

In [ ]:
X_train.loc[:, 'ord_2_WoE_Encode'] = X_train['ord_2'].map(woe_df['WoE'])
X_test.loc[:, 'ord_2_WoE_Encode'] = X_train['ord_2'].map(woe_df['WoE'])

### ord_3

In [ ]:
X_train['ord_3'].value_counts()

In [ ]:
a = pd.DataFrame(X_train['ord_3'].value_counts())

In [ ]:
a

In [ ]:
b=pd.DataFrame(train[train['target'] == 1]['ord_3'].value_counts())

In [ ]:
b

In [ ]:
ord_3_df = pd.concat([a,b], axis=1)

In [ ]:
ord_3_df['ord_3_full'] = ord_3_df.iloc[:,0]

In [ ]:
ord_3_df['ord_3_sub'] = ord_3_df.iloc[:,1]

In [ ]:
ord_3_df['비율'] = ord_3_df['ord_3_sub']/ord_3_df['ord_3_full']

In [ ]:
ord_3_df['비율'].sort_values()

- 알파벳 순서로 뒤로 갈수록 target이 1인 경우가 많다는 것을 확인
- 순서가 의미가 있는 Ordinal-encoding

In [ ]:
# Mapping을 활용환 Ordinal Encoding
alpha_dict = {"a": 1, "b": 2, "c": 3, "d": 4, 'e':5, 'f':6, 'g':7, 'h':8, 'i':9, 'j':10, 'k':11, 'l':12, 'm':13, 'n':14, 'o':15}
X_train["ord_3_ordinal"] = X_train.ord_3.map(alpha_dict)
X_test["ord_3_ordinal"] = X_test.ord_3.map(alpha_dict)

### ord_4

In [ ]:
X_train['ord_4'].value_counts()

In [ ]:
a = pd.DataFrame(X_train['ord_4'].value_counts())

In [ ]:
b=pd.DataFrame(train[train['target'] == 1]['ord_4'].value_counts())

In [ ]:
ord_4_df = pd.concat([a,b], axis=1)
ord_4_df['ord_4_full'] = ord_4_df.iloc[:,0]
ord_4_df['ord_4_sub'] = ord_4_df.iloc[:,1]
ord_4_df['비율'] = ord_4_df['ord_4_sub']/ord_4_df['ord_4_full']
ord_4_df['비율'].sort_values()

- 거의 알파벳순임. 순서가 의미가 있는 Ordinal-encoding helmert가 의미가 있답니다.

In [ ]:
alpha_dict = {"A": 1, "B": 2, "C": 3, "D": 4, 'E':5, 'F':6, 'G':7, 'H':8, 'I':9, 'J':10, 'K':11, 'L':12, 'M':13, 'N':14, 'O':15,
             'P':16, 'Q':17, 'R':18, 'S':19, 'T':20, 'U':21, 'V':22, 'W':23, 'X':24, 'Y':25, 'Z':26}
X_train["ord_4_ordinal"] = X_train.ord_4.map(alpha_dict)
X_test["ord_4_ordinal"] = X_test.ord_4.map(alpha_dict)

### ord_5

In [ ]:
X_train['ord_5'].value_counts()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

X_train_oe, X_test_oe, = X_train.copy(), X_test.copy()

le = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1, dtype=int) #학습데이터에 없는컬럼은 -1로 채워버리겠다.
X_train_oe['ord_5'] = le.fit_transform(X_train['ord_5'])
X_test_oe['ord_5'] = le.transform(X_test['ord_5'])

X_train_oe[cat]

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)